In [15]:
import sklearn
import shap
import numpy as np
import pandas as pd
#Evalutation metrics
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
#Keeping track of time
from fastprogress.fastprogress import progress_bar
import matplotlib as plt
import copy


In [3]:
#Churn dataset
url = 'https://github.com/EpistasisLab/pmlb/blob/master/datasets/churn/churn.tsv.gz?raw=true'
churn = pd.read_csv(url, compression='gzip', 
                                 header=0, sep = '\t', quotechar='"')

split = sklearn.model_selection.train_test_split(churn, shuffle=True)
X = split[0].drop('target', axis = 1)
Y = split[0].iloc[:, -1]
x = split[1].drop('target', axis = 1)
y = split[1].iloc[:, -1]

#This data comes from:
#Le, Trang T., William La Cava, Joseph D. Romano, John T. Gregg, Daniel J. Goldberg, Praneel Chakraborty, 
#Natasha L. Ray, Daniel Himmelstein, Weixuan Fu, and Jason H. Moore. 
#PMLB v1. 0: an open source dataset collection for benchmarking machine learning methods. arXiv preprint arXiv:2012.00058 (2020)


In [ ]:
clf = sklearn.ensemble.RandomForestClassifier()
clf = clf.fit(X,Y)
expl = shap.explainers.Tree(clf, X)
shap_vals = expl(x, check_additivity=False)
shap_vals[...,1]

In [7]:
# explainer = shap.explainers.Permutation(clf.predict, X)
# values = explainer(x)
# print(np.mean(values.values[...,1], axis = 0))
ranks = np.argsort(np.mean(np.abs(shap_vals.values[:,:,1]), axis = 0))
int(np.round(len(ranks)*0.10))
ranks[len(ranks)-2:len(ranks)]
ranks

array([ 2,  0, 14,  8, 11,  1,  3, 16, 15, 18, 13,  6,  5, 17, 10, 12,  4,
        9,  7, 19])

In [8]:
def train(clf, X, Y):
    accu = np.array([])
    accu_balanced = np.array([])
    f1 = np.array([])
    #Iterate through the datasets and retrain for each
    temp = copy.deepcopy(clf)

    #Fit model
    temp.fit(X, Y)
    
    return temp

In [9]:
def metrics(clf, x, y):   
    #Get metrics: 
    yhat = clf.predict(x)

    accu = accuracy_score(y, yhat)
    accu_balanced = balanced_accuracy_score(y, yhat)
    f1 = f1_score(y, yhat)
        
    return np.array([[accu], [accu_balanced], [f1]])

In [10]:
def explain(clf, X, x, explainer = shap.explainers.Permutation):
    if explainer != shap.explainers.Tree:
        # #Build Explanation
        explanation = explainer(clf.predict, X)
        shap_values = explanation(x)
    else:
        # #Build Explanation
        explanation = explainer(clf, X)
        shap_values = explanation(x, check_additivity = False)
    #Get SHAP values for positive class
    shap_values = shap_values.values[...,1]
    shap_values = np.abs(shap_values)
    shap_values = np.mean(shap_values, axis= 0)
    #Get ranks
    ranks = np.argsort(shap_values)
    ranks
    return ranks

In [21]:
np.zeros((3,1))

array([[0.],
       [0.],
       [0.]])

In [22]:
## Mask top features
def remove_top(t, rankings, X, Y, x, y, clf, base = np.empty((3,1))):
    
    #Make copies of our data to modify
    X_train = copy.deepcopy(X)
    x_test = copy.deepcopy(x)
    results = copy.deepcopy(base)

    #Set masking schedule
    j = int(np.round(len(rankings)*t))
    i = 0
    k = j
    
    #Mask and retrain
    while k <= len(rankings)+1:
        #Mask
        X_train.iloc[:, rankings[i:k]] = X_train.iloc[:, rankings[i:k]].mean()
        x_test.iloc[:, rankings[i:k]] = x_test.iloc[:, rankings[i:k]].mean() 
        #Retrain
        model = train(clf, X_train, Y)
        results =  np.hstack((results, metrics(model, x_test, y)))
        
        #Move iterator forward
        i += j
        k += j

    return results

remove_top(0.10, ranks, X, Y, x, y, clf)

array([[0.        , 0.9512    , 0.9544    , 0.9512    , 0.9544    ,
        0.944     , 0.9384    , 0.9024    , 0.8384    , 0.8224    ,
        0.8632    ],
       [0.        , 0.84624327, 0.86039976, 0.85116444, 0.85793918,
        0.83715158, 0.83636842, 0.76138291, 0.68740278, 0.65106851,
        0.5       ],
       [0.        , 0.79734219, 0.81433225, 0.8       , 0.81311475,
        0.77124183, 0.75555556, 0.61392405, 0.44808743, 0.39010989,
        0.        ]])

In [23]:
## Mask bottom features
def remove_bottom(t, rankings, X, Y, x, y, clf, base = np.empty((3,1))):
    #Make copies of our data to modify
    X_train = copy.deepcopy(X)
    x_test = copy.deepcopy(x)
    results = copy.deepcopy(base)
    
    
    #Set masking schedule
    j = int(np.round(len(rankings)*t))
    i = len(rankings) - j
    k = len(rankings)
    
    #Mask and retrain
    while k >= j:
        #Mask
        X_train.iloc[:, rankings[i:k]] = X_train.iloc[:, rankings[i:k]].mean()
        x_test.iloc[:, rankings[i:k]] = x_test.iloc[:, rankings[i:k]].mean() 
            
        #Retrain
        model = train(clf, X_train, Y)
        results =  np.hstack((results, metrics(model, x_test, y)))
        
        
        #Move iterator forward
        i -= j
        k -= j

    return results


remove_bottom(0.10, ranks, X, Y, x, y, clf)

array([[0.8632    , 0.9272    , 0.8624    , 0.8608    , 0.8624    ,
        0.8624    , 0.8616    , 0.8624    , 0.8544    , 0.8616    ,
        0.8632    ],
       [0.5       , 0.73637871, 0.49953661, 0.49860982, 0.49953661,
        0.49953661, 0.49907322, 0.49953661, 0.50474503, 0.50645497,
        0.5       ],
       [0.        , 0.64031621, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.04210526, 0.03351955,
        0.        ]])

In [25]:
## Mask top features
def remove_random(t, rankings, X, Y, x, y, clf, base = np.empty((3,1))):
    
    random_choices = np.random.choice(len(rankings), len(rankings), replace = False)

    #Make copies of our data to modify
    X_train = copy.deepcopy(X)
    x_test = copy.deepcopy(x)
    results = copy.deepcopy(base)
    
    #Set masking schedule
    j = int(np.round(len(random_choices)*t))
    i = 0
    k = j
    
    #Mask and retrain
    while k <= len(random_choices):
        #Mask
        X_train.iloc[:, random_choices[i:k]] = X_train.iloc[:, random_choices[i:k]].mean()
        x_test.iloc[:, random_choices[i:k]] = x_test.iloc[:, random_choices[i:k]].mean() 
            
        #Retrain
        model = train(clf, X_train, Y)
        results = np.hstack((results, metrics(model, x_test, y)))


        #Move iterator forward
        i += j
        k += j

    return results

remove_random(0.10, ranks, X, Y, x, y, clf)

array([[6.17582057e-321, 9.45600000e-001, 9.31200000e-001,
        9.16000000e-001, 9.10400000e-001, 8.91200000e-001,
        8.80800000e-001, 8.81600000e-001, 8.40800000e-001,
        8.28000000e-001, 8.63200000e-001],
       [6.17582057e-321, 8.23314852e-001, 7.85446780e-001,
        7.52036486e-001, 7.26647481e-001, 6.41708535e-001,
        6.15999762e-001, 6.16463154e-001, 5.36236715e-001,
        5.48507119e-001, 5.00000000e-001],
       [6.17582057e-321, 7.67123288e-001, 6.99300699e-001,
        6.31578947e-001, 5.91240876e-001, 4.28571429e-001,
        3.65957447e-001, 3.67521368e-001, 1.67364017e-001,
        2.06642066e-001, 0.00000000e+000]])

In [26]:
def roar(X, Y, x, y, clf, explainer = shap.explainers.Permutation, t = 0.10, repeats = 10):
    #Initialize the frames
    model = train(clf, X, Y)
    base = metrics(model, x, y)
    #Initialize
    ranks = explain(model, X, x, explainer)
     
    top = remove_top(t, ranks, X, Y, x, y, clf, base)
    print("Top")
    bottom = remove_bottom(t, ranks, X, Y, x, y, clf, base)
    print("Bottom")
    random = remove_bottom(t, ranks, X, Y, x, y, clf, base)
    print("Random")

    #Set progress bar
    mb = progress_bar(range(repeats - 1))
    #Repeat x times
    for i in mb:
        ranks = explain(model, X, x, explainer)
        top = np.dstack((top, remove_top(t, ranks, X, Y, x, y, clf, base)))
        bottom = np.dstack((bottom, remove_bottom(t, ranks, X, Y, x, y, clf, base)))
        random = np.dstack((random, remove_bottom(t, ranks, X, Y, x, y, clf, base)))
    return np.array([top, bottom, random])

In [27]:
test = roar(X, Y, x, y, clf = sklearn.ensemble.RandomForestClassifier(), explainer = shap.explainers.Tree, t = 0.10, repeats = 10)

 98%|===================| 2451/2500 [00:25<00:00]        

Explained
Top
Bottom
Random


 99%|===================| 2463/2500 [00:25<00:00]        

In [60]:
#ROAR returns an array of the following shape (3, metrics, iterations, steps)
#shape[0] is top, bottom, random
#shape[1] is the surveyed metrics, in this case: accuracy, balanced accuracy, f1
#shape[2] is the number of iterations ran
#shape[3] is the # of steps that had to be done to mask all of the features
print(test.shape)

#np.mean(roar_results, axis = 2) returns the average scores over all repeats.
print(np.mean(test, axis = 2).shape)

results_mean = np.mean(test, axis = 2)
auc = np.trapz(results_mean)
print(auc)
diff = auc[1] - auc[2]
print(diff)





(3, 3, 11, 10)
(3, 3, 10)
[[8.24567273 6.99287495 5.74537706]
 [7.88218182 4.9853168  1.23576105]
 [7.88272727 4.98272479 1.22499091]]
[-0.00054545  0.00259201  0.01077013]


array([-0.04408263, -0.28708624, -0.78491211])

In [61]:
#Returns the ROAR score for each value
def roar_score(results):
    results_mean = np.mean(test, axis = 2)
    auc = np.trapz(results_mean)
    score = np.where(auc[1] - auc[0] < 0,  (auc[1] - auc[0])/auc[0], (auc[1] - auc[0])/auc[1] )
    return score

In [62]:
roar_score(test)

array([-0.04408263, -0.28708624, -0.78491211])